## Importing libraries

In [1]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [2]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.13.0


In [3]:
import pandas as pd
import seaborn as sns

from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Load dataset
categories = np.load('training_dataset_preprocessed/categories_preprocessed.npy')
data_train = np.load('training_dataset_preprocessed/training_data_preprocessed.npy')
valid_periods = np.load('training_dataset_preprocessed/valid_periods_preprocessed.npy')

In [5]:
def normalize_window(windows_batch, return_minmax= False):
    single_sample = False
    if len(windows_batch.shape) == 2:
        single_sample = True

    # Squeeze last dimension
    windows_batch = np.squeeze(windows_batch)
    batch_size = len(windows_batch)

    # For each sample compute min and max value
    if single_sample:
        minmax_batch_array = np.array([[np.min(windows_batch)], [np.max(windows_batch)]])
    else:
        minmax_batch_array = np.array([[np.min(windows_batch[i]), np.max(windows_batch[i])] for i in range(batch_size)])
   
    # Perform min-max scaling for each sample
    if single_sample:
        scaled_batch = np.array([(windows_batch - minmax_batch_array[0]) / (minmax_batch_array[1] - minmax_batch_array[0])])
    else:
        scaled_batch = np.array([(windows_batch[i] - minmax_batch_array[i, 0]) / (minmax_batch_array[i, 1] - minmax_batch_array[i, 0]) for i in range(batch_size)])

    # Return results
    if return_minmax:
        return scaled_batch, minmax_batch_array
    else:
        return scaled_batch

In [6]:
data_train.shape

(47974, 2776, 1)

In [7]:
data_try = data_train[:3, -10:, :]
data_try.shape

(3, 10, 1)

In [8]:
data_try =np.squeeze(data_try)

In [9]:
data_try.shape

(3, 10)

In [10]:
len(data_try)

3

In [11]:
data_try[0]

array([0.99555146, 0.82870984, 0.80936688, 0.82870984, 0.74328417,
       0.80936688, 0.60750486, 0.74328417, 0.62639178, 0.7536182 ])

In [12]:
min_max_batch_array = np.array([[np.min(data_try), np.max(data_try)]])
min_max_batch_array

array([[0.27479158, 1.        ]])

In [13]:
min_max_batch_array.shape

(1, 2)

In [14]:
data_try.shape

(3, 10)

In [15]:
data_try[2]

array([1.        , 0.54478243, 0.50786404, 0.47132041, 0.36748344,
       0.65839054, 0.3196237 , 0.54485458, 0.53497663, 0.45541526])

In [16]:
min_max_batch_array[2,0]

IndexError: index 2 is out of bounds for axis 0 with size 1

In [ ]:
data_try[2] - min_max_batch_array[2,0]

array([0.6803763 , 0.22515874, 0.18824034, 0.15169672, 0.04785975,
       0.33876684, 0.        , 0.22523088, 0.21535293, 0.13579156])

In [ ]:
scaled_batch = np.array([(data_try[i] - min_max_batch_array[i,0]) / (min_max_batch_array[i,1] - min_max_batch_array[i,0]) for i in range(len(data_try))])
scaled_batch

array([[1.        , 0.57004745, 0.52020045, 0.57004745, 0.34990465,
        0.52020045, 0.        , 0.34990465, 0.04867178, 0.37653554],
       [1.        , 0.58326807, 0.33853142, 0.58326807, 0.33751611,
        0.33853142, 0.        , 0.33751611, 0.20274657, 0.34324533],
       [1.        , 0.33093266, 0.27667093, 0.22296002, 0.07034305,
        0.497911  , 0.        , 0.33103869, 0.31652033, 0.19958303]])

In [ ]:
scaled_batch.shape

(3, 10)

In [ ]:
data_try = data_train[:3, -10:, :]
data_try.shape

(3, 10, 1)

In [ ]:
new_data, minmax = normalize_window(data_try, return_minmax= True)

In [ ]:
data_try

array([[[0.99555146],
        [0.82870984],
        [0.80936688],
        [0.82870984],
        [0.74328417],
        [0.80936688],
        [0.60750486],
        [0.74328417],
        [0.62639178],
        [0.7536182 ]],

       [[1.        ],
        [0.69778249],
        [0.52029741],
        [0.69778249],
        [0.5195611 ],
        [0.52029741],
        [0.27479158],
        [0.5195611 ],
        [0.4218251 ],
        [0.52371598]],

       [[1.        ],
        [0.54478243],
        [0.50786404],
        [0.47132041],
        [0.36748344],
        [0.65839054],
        [0.3196237 ],
        [0.54485458],
        [0.53497663],
        [0.45541526]]])

In [ ]:
new_data

array([[1.        , 0.57004745, 0.52020045, 0.57004745, 0.34990465,
        0.52020045, 0.        , 0.34990465, 0.04867178, 0.37653554],
       [1.        , 0.58326807, 0.33853142, 0.58326807, 0.33751611,
        0.33853142, 0.        , 0.33751611, 0.20274657, 0.34324533],
       [1.        , 0.33093266, 0.27667093, 0.22296002, 0.07034305,
        0.497911  , 0.        , 0.33103869, 0.31652033, 0.19958303]])

In [ ]:
new_data.shape

(3, 10)

In [ ]:
minmax

array([[0.60750486, 0.99555146],
       [0.27479158, 1.        ],
       [0.3196237 , 1.        ]])

In [ ]:
from Utils import normalize_window, inverse_normalize_window

In [ ]:
back = inverse_normalize_window(new_data, minmax)

In [ ]:
back

array([[0.99555146, 0.82870984, 0.80936688, 0.82870984, 0.74328417,
        0.80936688, 0.60750486, 0.74328417, 0.62639178, 0.7536182 ],
       [1.        , 0.69778249, 0.52029741, 0.69778249, 0.5195611 ,
        0.52029741, 0.27479158, 0.5195611 , 0.4218251 , 0.52371598],
       [1.        , 0.54478243, 0.50786404, 0.47132041, 0.36748344,
        0.65839054, 0.3196237 , 0.54485458, 0.53497663, 0.45541526]])

In [ ]:
back == np.squeeze(data_try)

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])

## Try as a layer

In [17]:
data_try = data_train[:3, -10:, :]
data_try.shape

(3, 10, 1)

In [21]:
from Utils import NormalizeWindowLayer, InverseNormalizeWindowLayer

In [19]:
output = NormalizeWindowLayer(return_minmax= True) (data_try)

In [20]:
output

(<tf.Tensor: shape=(3, 10), dtype=float32, numpy=
 array([[1.        , 0.5700475 , 0.5202005 , 0.5700475 , 0.34990466,
         0.5202005 , 0.        , 0.34990466, 0.04867179, 0.3765355 ],
        [1.        , 0.5832681 , 0.33853143, 0.5832681 , 0.33751616,
         0.33853143, 0.        , 0.33751616, 0.2027466 , 0.34324533],
        [1.        , 0.33093268, 0.27667093, 0.22296003, 0.07034303,
         0.49791095, 0.        , 0.33103868, 0.31652027, 0.199583  ]],
       dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.60750484, 0.99555147],
        [0.27479157, 1.        ],
        [0.3196237 , 1.        ]], dtype=float32)>)

In [22]:
back = InverseNormalizeWindowLayer() (output)

In [23]:
back

<tf.Tensor: shape=(3, 10), dtype=float32, numpy=
array([[0.99555147, 0.82870984, 0.8093669 , 0.82870984, 0.74328417,
        0.8093669 , 0.60750484, 0.74328417, 0.62639177, 0.7536182 ],
       [1.        , 0.6977825 , 0.5202974 , 0.6977825 , 0.5195611 ,
        0.5202974 , 0.27479157, 0.5195611 , 0.4218251 , 0.523716  ],
       [1.        , 0.54478246, 0.50786406, 0.47132042, 0.36748344,
        0.6583905 , 0.3196237 , 0.5448546 , 0.5349766 , 0.45541525]],
      dtype=float32)>

In [24]:
data_try

array([[[0.99555146],
        [0.82870984],
        [0.80936688],
        [0.82870984],
        [0.74328417],
        [0.80936688],
        [0.60750486],
        [0.74328417],
        [0.62639178],
        [0.7536182 ]],

       [[1.        ],
        [0.69778249],
        [0.52029741],
        [0.69778249],
        [0.5195611 ],
        [0.52029741],
        [0.27479158],
        [0.5195611 ],
        [0.4218251 ],
        [0.52371598]],

       [[1.        ],
        [0.54478243],
        [0.50786404],
        [0.47132041],
        [0.36748344],
        [0.65839054],
        [0.3196237 ],
        [0.54485458],
        [0.53497663],
        [0.45541526]]])